Import the necessary packages.

### Load the necessary modules.

In [ ]:
import jax.numpy as jnp
import jax
from rheedium import *

#### Use `autoreload` to reload changed modules, you may disable this for normal use.

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
structure_file = inout.parse_cif("../data/SrTiO3.cif")

TypeError: scan body function carry input and carry output must have equal types (e.g. shapes and dtypes of arrays), but they differ:

  * the input carry component loop_carry[1][0] has type float64[1,3] but the corresponding output carry component has type float64[2,3], so the shapes do not match;
  * the input carry component loop_carry[1][1] has type float64[1,4] but the corresponding output carry component has type float64[2,4], so the shapes do not match.

Revise the function so that all output types (e.g. shapes and dtypes) match the corresponding input types.

In [ ]:
structure_file

In [ ]:
zone_axis = jnp.array([1, 1, 1])

In [ ]:
grazed_crystal = ucell.atom_scraper(structure_file, zone_axis, 3)

In [ ]:
grazed_crystal

CrystalStructure(frac_positions=Array([[ 0.5,  0.5,  0.5, 38. ],
       [ 0. ,  0. ,  0.5,  8. ]], dtype=float64), cart_positions=Array([[1.97256507e+00, 1.97256507e+00, 1.97256506e+00, 3.80000000e+01],
       [1.20784775e-16, 1.20784775e-16, 1.97256506e+00, 8.00000000e+00]],      dtype=float64), cell_lengths=Array([3.79157414, 3.79157414, 3.79157414], dtype=float64), cell_angles=Array([94.74025632, 94.74025632, 94.74025632], dtype=float64))

In [ ]:
simul.simulate_rheed_pattern(structure_file, jnp.asarray(10))

TypeError: missing a required argument: 'c'